In [42]:
import math
import copy
fread = open("Session 15.txt", "r") #netdata.txt
G_v_e = fread.readline()
G_v_e = G_v_e.split(' ') #change to \t if tab spaced
V = int(G_v_e[0])
E = int(G_v_e[1].strip())
V_list = list(range(0,V,1))
lines = fread.readlines()
A_dict = {}
G_mat = [ [ 0 for i in range(4) ] for j in range(E) ]
g = 0
for line in lines:
    t_h = line.split(' ') #change to \t if tab spaced
    print(t_h)
    G_mat[g][0] = int(t_h[0])
    G_mat[g][1] = int(t_h[1])
    G_mat[g][2] = int(t_h[2])
    G_mat[g][3] = int(t_h[3].strip())
    g += 1
for i in range(E):
    tail = G_mat[i][0]
    head = G_mat[i][1]
    if tail in A_dict.keys():
        A_dict[tail].append(head)        
    else:
        A_dict[tail] = [head]

capacity = [[0 for i in range(V)] for j in range(V)]
cost = [[math.inf for i in range(V)] for j in range(V)]
for i in range(E):
    tail = G_mat[i][0]
    head = G_mat[i][1]
    cost[tail][head] = G_mat[i][2]
    capacity[tail][head] = G_mat[i][3]

print(G_mat, "\n")
print(A_dict)
print(cost)
print(capacity)
fread.close()

['0', '1', '4', '3\n']
['0', '3', '3', '4\n']
['0', '4', '5', '6\n']
['1', '2', '4', '2\n']
['1', '6', '6', '5\n']
['2', '7', '3', '4\n']
['3', '1', '2', '3\n']
['4', '0', '8', '2\n']
['4', '2', '2', '3\n']
['4', '3', '2', '4\n']
['4', '5', '7', '5\n']
['5', '7', '1', '2\n']
['6', '2', '1', '5\n']
['6', '7', '1', '4\n']
[[0, 1, 4, 3], [0, 3, 3, 4], [0, 4, 5, 6], [1, 2, 4, 2], [1, 6, 6, 5], [2, 7, 3, 4], [3, 1, 2, 3], [4, 0, 8, 2], [4, 2, 2, 3], [4, 3, 2, 4], [4, 5, 7, 5], [5, 7, 1, 2], [6, 2, 1, 5], [6, 7, 1, 4]] 

{0: [1, 3, 4], 1: [2, 6], 2: [7], 3: [1], 4: [0, 2, 3, 5], 5: [7], 6: [2, 7]}
[[inf, 4, inf, 3, 5, inf, inf, inf], [inf, inf, 4, inf, inf, inf, 6, inf], [inf, inf, inf, inf, inf, inf, inf, 3], [inf, 2, inf, inf, inf, inf, inf, inf], [8, inf, 2, 2, inf, 7, inf, inf], [inf, inf, inf, inf, inf, inf, inf, 1], [inf, inf, 1, inf, inf, inf, inf, 1], [inf, inf, inf, inf, inf, inf, inf, inf]]
[[0, 3, 0, 4, 6, 0, 0, 0], [0, 0, 2, 0, 0, 0, 5, 0], [0, 0, 0, 0, 0, 0, 0, 4], [0, 3, 0, 0, 

In [43]:
def create_residual_network1(A_dict, capacity, cost, V):
    R = []
    enode = V
    visited = [[0 for i in range(V)] for j in range(V)]
    for i in A_dict.keys():
        for j in A_dict[i]:
            #print(i, j)
            if (j in A_dict.keys()):
                if (visited[i][j] == 0):
                    if (i in A_dict[j]):
                        #add two nodes
                        R.append([i, enode, cost[i][j], capacity[i][j]])
                        #R.append([enode, i, capacity[i][j], cost[i][j]])
                        R.append([enode, j, 0, capacity[i][j]])
                        enode = enode + 1
                        R.append([j, enode, cost[j][i], capacity[j][i]])
                        R.append([enode, i, 0, capacity[j][i]]) 
                        enode = enode + 1
                    else:
                        R.append([i, j,  cost[i][j], capacity[i][j]])
                else:
                    continue
            else:
                R.append([i, j,  cost[i][j], capacity[i][j]])
            #print(R)
            visited[i][j] = 1
            visited[j][i] = 1
    #print(R)
    itr = len(R)
    #print(itr)
    for i in range(itr):
        t = R[i][0]
        h = R[i][1]
        ncost = -1*R[i][2]
        R.append([h, t, ncost, 0])
    return R, enode

In [44]:
def retrieve_flow_1(R_orig, R_capacity):
    flow = []
    for i in range(len(R_orig)):
        t = R_orig[i][0]
        h = R_orig[i][1]
        old = R_orig[i][3]
        new = R_capacity[t][h]
        if (old > new):
            flow.append(old - new)
        else:
            flow.append(0)            
    return flow


# Develop modified dictionary
def mod_dict(R, capacity):
    R_dict = {}
    for e in range(len(R)):
        t = R[e][0]
        h = R[e][1]
        if (capacity[t][h] != 0):
            if (t in R_dict.keys()):
                if (h not in R_dict[t]):
                    R_dict[t].append(h)
            else:
                R_dict[t] = [h]
        else:
            if (t in R_dict.keys()):
                if (h in R_dict[t]):
                    R_dict[t].remove(h)
    return R_dict


In [ ]:
def cap_Dijkstra(A_dict, cost, V, s, t):
    parent = {}
    for i in range(V):
        parent[i] = i
    dist = [math.inf]*V
    dist[s] = 0
    fixed = [s]
    chosen = s
    while (t not in fixed):
        if (chosen not in A_dict.keys()):
            print("cant use DJ here")
            return dist[t], parent
        chosen_range = A_dict[chosen]
        k = -1
        for i in range(len(chosen_range)):
            j = chosen_range[i]
            if (j in fixed):
                continue
            if (dist[j] >= dist[chosen] + cost[chosen][j]):
                dist[j] = dist[chosen] + cost[chosen][j]
                parent[j] = chosen
        min_d = 1e8
        min_v = -1
        for v in range(V):
            if ((dist[v] <= min_d) and (v not in fixed) and (dist[v] != math.inf) and (v in A_dict.keys() or v == t)):
                min_d = dist[v]
                min_v = v
        k = min_v
        if (k == -1 or k == math.inf):
            break
        else:
            fixed.append(k)
            chosen = k
    return dist[t], parent

def create_residual_network(G_mat, A_dict, capacity):
    R = copy.deepcopy(G_mat)
    for i in A_dict.keys():
        for j in A_dict[i]:
            if (j in A_dict.keys()):
                if (i in A_dict[j]):
                    continue
            R.append([j, i, 0, 0])
    return R


def find_s_t_path(R, s, end, V, capacity): # Use R for nodes & arc connects and 'capacity' for residual capacity
    mod_cost = [[math.inf for i in range(V)] for j in range(V)]
    R_dict = {}
    
    #print('capacity')
    #for i in range(V):
    #    print(capacity[i])
        
    # Develop modified cost & dictionary
    for e in range(len(R)):
        t = R[e][0]
        h = R[e][1]
        if (capacity[t][h] != 0):
            mod_cost[t][h] = 1
            if (t in R_dict.keys()):
                if (h not in R_dict[t]):
                    R_dict[t].append(h)
            else:
                R_dict[t] = [h]
        else:
            if (t in R_dict.keys()):
                if (h in R_dict[t]):
                    R_dict[t].remove(h)
                    
    #Print values
    #print('modified cost')
    #for i in range(V):
    #    print(mod_cost[i])
    #print(R_dict)
    
    # Find shortest s-t path
    min_cost, parent = cap_Dijkstra(R_dict, mod_cost, V, s, end)
    #print(min_cost)
    #print(parent)
    path = []
    if (min_cost == math.inf):
        return path
    path = [end]
    curr = end
    while (curr != s and curr != parent[curr]):
        curr = parent[curr]
        path.insert(0, curr)
        #print(path)   
    return path

In [ ]:
def feasible_flow(R_orig, V, capacity, R_cost, s, end, f): #find one!!! - use FF kind of algo
    flow_value = 0
    flow_cost
    path = find_s_t_path(R, s, end, V, capacity)
    while (len(path)! = 0 and flow_value < f):
        min_cap = math.inf
        p_cost = 0
        for n in range(len(path) - 1):
            t = path[n]
            h = path[n+1]
            if (capacity[t][h] < min_cap):
                min_cap = capacity[t][h]
            p_cost = p_cost + R_cost[t][h]
        flow_value = flow_value + min_cap
        add_flow = min_cap
        if (flow_value > f):
            add_flow = f - (flow_value - min_cap)
        flow_cost = flow_cost + p_cost
        for n in range(len(path)-1):
            t = path[n]
            h = path[n+1]
            capacity[t][h] = capacity[t][h] - min_cap
            capacity[h][t] = capacity[h][t] + min_cap
        path = find_s_t_path(R, s, end, V, capacity)
    flow = retrieve_flow_1(R_orig, R_capacity)
    
return flow, flow_cost

def find_negative_cycle(parent, diff_dist): #in notes
    cn = []
    for i in range(len(diff_dist)):
        if (diff_dist[i] != 0):
            cn.append(i)
    n = cn[0]
    path = [n]
    nxt = parent[n]
    while (nxt not in path):
        path.append(nxt)
        nxt = parent[nxt]
    if (nxt == n):
        return path
    else:
        return path[1:]

In [ ]:
# normal Bellman Ford
def m_Bellman_Ford(R, V, E, s, t, R_cost, R_capacity):
    parent = {}
    for i in range(V):
        parent[i] = i
    #print(parent)
    dist = [math.inf]*V
    prev_dist = [math.inf]*V
    dist[s] = 0
    changed_flag = True
    itr = 1
    while (changed_flag):
        changed_flag = False
        d_copy = dist.copy()  #remove this for modified
        prev_dist = dist.copy()
        for i in range(E):
            tail = R[i][0]
            head = R[i][1]
            if (d_copy[head] > dist[tail] + R_cost[tail][head] and R_capacity[tail][head] > 0): #d_copy[head] to dist[head]
                d_copy[head] = dist[tail] + R_cost[tail][head] #d_copy[head] to dist[head]
                parent[head] = tail
                changed_flag = True
        dist = d_copy #remove this for modified
        #print(itr)
        #print(dist)
        #print(parent)
        itr = itr + 1
        if (itr > V): #to take care of negative cycles
            break        
    return dist, prev_dist, parent

In [1]:
#cycle cancelling
def cycle_cancel(R, V, R_capacity, R_cost, s, t, f):
    R_orig = copy.deepcopy(R)
    flow, flow_cost = feasible_flow(R_orig, V, R_capacity, R_cost, s, t, f)
    dist, prev_dist, parent = m_Bellman_Ford(R, Vnew, len(R), s, t, R_cost)
    diff_dist = dist-prev_dist
    n_cycle = find_negative_cycle(parent, diff_dist)
    while (len(n_cycle)!= 0):
        n_cost = 0
        cmin = math.inf
        for i in range(len(n_cycle)-1):
            tail = n_cycle[i]
            head = n_cycle[i+1]
            n_cost = n_cost + R_cost[tail][head]
            if (R_capacity[tail][head] < cmin):
                cmin = R_capacity[tail][head]
        flow_cost = flow_cost - cmin*n_cost
        
        #update capacity
        for i in range(len(path)-1):
            tail = path[i]
            head = path[i+1]
            R_capacity[tail][head] =  R_capacity[tail][head] - cmin
            R_capacity[head][tail] =  R_capacity[head][tail] + cmin
            
        dist, prev_dist, parent = m_Bellman_Ford(R, Vnew, len(R), s, t, R_cost, R_capacity)
        diff_dist = dist-prev_dist
        n_cycle = find_negative_cycle(parent, diff_dist)
        
    flow = retrieve_flow_1(R_orig, R_capacity)
    return flow, flow_cost
    

In [47]:
s = 0
t = 7
capacity = [[0 for i in range(V)] for j in range(V)]
cost = [[math.inf for i in range(V)] for j in range(V)]
for i in range(E):
    tail = G_mat[i][0]
    head = G_mat[i][1]
    cost[tail][head] = G_mat[i][2]
    capacity[tail][head] = G_mat[i][3]
    
R, Vnew = create_residual_network1(A_dict, capacity, cost, V)

#print(Vnew)
R_capacity = [[0 for i in range(Vnew)] for j in range(Vnew)]
R_cost = [[math.inf for i in range(Vnew)] for j in range(Vnew)]
for i in range(len(R)):
    tail = R[i][0]
    head = R[i][1]
    R_cost[tail][head] = R[i][2]
    R_capacity[tail][head] = R[i][3]

for i in range(len(R)):
    print(R[i])
R_dict = mod_dict(R, R_capacity)
print(R_dict)

[0, 1, 4, 3]
[0, 3, 3, 4]
[0, 8, 5, 6]
[8, 4, 0, 6]
[4, 9, 8, 2]
[9, 0, 0, 2]
[1, 2, 4, 2]
[1, 6, 6, 5]
[2, 7, 3, 4]
[3, 1, 2, 3]
[4, 2, 2, 3]
[4, 3, 2, 4]
[4, 5, 7, 5]
[5, 7, 1, 2]
[6, 2, 1, 5]
[6, 7, 1, 4]
[1, 0, -4, 0]
[3, 0, -3, 0]
[8, 0, -5, 0]
[4, 8, 0, 0]
[9, 4, -8, 0]
[0, 9, 0, 0]
[2, 1, -4, 0]
[6, 1, -6, 0]
[7, 2, -3, 0]
[1, 3, -2, 0]
[2, 4, -2, 0]
[3, 4, -2, 0]
[5, 4, -7, 0]
[7, 5, -1, 0]
[2, 6, -1, 0]
[7, 6, -1, 0]
{0: [1, 3, 8], 8: [4], 4: [9, 2, 3, 5], 9: [0], 1: [2, 6], 2: [7], 3: [1], 5: [7], 6: [2, 7]}


In [48]:
f = 10 #input flow
flow, flow_cost = cycle_cancel(R, Vnew, R_capacity, R_cost, s, t, f)
# Flow_cost
print('Total flow cost: ', flow_cost)
# Flow thru arcs
print('Flow through arcs-')
flow_value = 0
for i in range(len(R)):
    print('(', R[i][0], ',' , R[i][1], ')', ':', flow[i])

Total flow cost:  113
Flow through arcs-
( 0 , 1 ) : 3
( 0 , 3 ) : 2
( 0 , 8 ) : 5
( 8 , 4 ) : 5
( 4 , 9 ) : 0
( 9 , 0 ) : 0
( 1 , 2 ) : 1
( 1 , 6 ) : 4
( 2 , 7 ) : 4
( 3 , 1 ) : 2
( 4 , 2 ) : 3
( 4 , 3 ) : 0
( 4 , 5 ) : 2
( 5 , 7 ) : 2
( 6 , 2 ) : 0
( 6 , 7 ) : 4
( 1 , 0 ) : 0
( 3 , 0 ) : 0
( 8 , 0 ) : 0
( 4 , 8 ) : 0
( 9 , 4 ) : 0
( 0 , 9 ) : 0
( 2 , 1 ) : 0
( 6 , 1 ) : 0
( 7 , 2 ) : 0
( 1 , 3 ) : 0
( 2 , 4 ) : 0
( 3 , 4 ) : 0
( 5 , 4 ) : 0
( 7 , 5 ) : 0
( 2 , 6 ) : 0
( 7 , 6 ) : 0
